In [1]:
import tensorflow as tf
from tensorflow import keras
print("TensorFlow version:", tf.__version__)

2022-11-05 00:17:58.823499: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


TensorFlow version: 2.4.1


In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, cohen_kappa_score
import matplotlib as mpl
import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
from os.path import join
import os

In [4]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [5]:
data_folder = "../../data/preprocessed/"

In [10]:
X = pd.read_pickle(join(data_folder, "X.pk"))
y = pd.read_pickle(join(data_folder, "Y.pk"))
X_mat = X.values
y_vec = y.values.flatten()
scaler = StandardScaler()
scaler.fit(X_mat)
X_mat = scaler.transform(X_mat)

x_train, x_test, y_train, y_test = train_test_split(X_mat, y_vec, test_size=0.12, random_state=123)


In [14]:
np.sum(y_test == 2)

7899

In [11]:
x_test.shape

(8486, 262)

In [15]:
EPOCHS = 100
BATCH_SIZE = 2048


In [50]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3)
])

In [51]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.17441374, -0.37345812,  0.5108738 ]], dtype=float32)

In [52]:
tf.nn.softmax(predictions).numpy()

array([[0.33577758, 0.19413953, 0.47008285]], dtype=float32)

In [53]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# sample_weight=tf.constant([2, 2, 0.1])
loss_fn(y_train[:1], predictions, sample_weight=tf.constant([2, 2, 0.1])).numpy()

2.2402103

In [64]:
model.compile(optimizer='adam',
              # loss=loss_fn,
              loss=loss_fn,
              metrics=['categorical_accuracy'])

In [65]:
class_weight = {0:1.2,
               1:1.2,
               2:0.1}
model.fit(x_train, y_train, epochs=100, class_weight=class_weight)

Epoch 1/100
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1260 - categorical_accuracy: 0.0738
Epoch 2/100
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1182 - categorical_accuracy: 0.0634
Epoch 3/100
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1179 - categorical_accuracy: 0.0716
Epoch 4/100
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1164 - categorical_accuracy: 0.0684
Epoch 5/100
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1145 - categorical_accuracy: 0.0672
Epoch 6/100
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1156 - categorical_accuracy: 0.0762
Epoch 7/100
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1144 - categorical_accuracy: 0.0704
Epoch 8/100
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1155 - categorical_accuracy: 0.0712
Epoch 9/100
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1

In [66]:
model.evaluate(x_test,  y_test, verbose=2)



266/266 - 0s - loss: 0.7958 - categorical_accuracy: 0.0350


[0.7958359122276306, 0.0349988229572773]

In [67]:
logits = model.predict(x_test)
results = tf.math.argmax(tf.nn.softmax(logits).numpy(), axis =1)

In [68]:
cm = confusion_matrix(results, y_test)
score2 = cohen_kappa_score(results, y_test)

In [69]:
cm

array([[  17,   18,  262],
       [  26,   43,  618],
       [ 196,  287, 7019]])

In [70]:
score2

0.04050494385213421

## New Predictions

In [71]:
X = pd.read_pickle(join(data_folder, "test.pk"))
X_mat = X.values

In [72]:
X_mat = scaler.transform(X_mat)

In [74]:
logits = model.predict(X_mat)
results = tf.math.argmax(tf.nn.softmax(logits).numpy(), axis =1)

In [79]:
predicted_vals = results.numpy()

In [80]:

report = pd.DataFrame({'Id': X.index, 'pred': predicted_vals})
report.to_csv("submission.csv", index=False)